In [1]:
# customer_segmentation.py
import psycopg2
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import joblib
from sqlalchemy import create_engine
import io
from airflow import DAG
from airflow.operators.python_operator import PythonOperator
from airflow.operators.email_operator import EmailOperator
from datetime import datetime, timedelta

c:\Users\henri\AppData\Local\Programs\Python\Python311\Lib\site-packages\airflow\__init__.py:45: RuntimeWarning: Airflow currently can be run on POSIX-compliant Operating Systems. For development, it is regularly tested on fairly modern Linux Distros and recent versions of macOS. On Windows you can run it via WSL2 (Windows Subsystem for Linux 2) or via Linux Containers. The work to add Windows support is tracked via https://github.com/apache/airflow/issues/10388, but it is not a high priority.
  warnings.warn(


AttributeError: module 'os' has no attribute 'register_at_fork'

# Conectando com o Banco de Dados

In [ ]:
conn = psycopg2.connect(
    host="fidelidademaisibm.cb4g0aguevt3.us-east-2.rds.amazonaws.com",
    database="ibm",
    user="thiago",
    password="ibmteste",
    port=5432
)

cursor = conn.cursor()
cursor.execute("SELECT version();")
print(cursor.fetchone())

('PostgreSQL 17.4 on aarch64-unknown-linux-gnu, compiled by gcc (GCC) 12.4.0, 64-bit',)


# Listando Todas as Tabelas e Suas Colunas

In [ ]:
cursor.execute("""
    SELECT table_name
    FROM information_schema.tables
    WHERE table_schema = 'public'
""")
tables = cursor.fetchall()
print("Tabelas disponíveis:")
for t in tables:
    print(t[0])

Tabelas disponíveis:
ibm_rfm
ibm_familia
ibm_grupo
ibm_grupo_marca
ibm_marca
ibm_produto
ibm_itens
ibm_linha
ibm_linha_subgrupo
ibm_subgrupo
ibm_cliente
nota_fiscal
ibm_operacao
ibm_vendedor
modelos_ml


In [ ]:
# Função para listar colunas de cada tabela
def listar_colunas(conn, tabela):
    query = f"""
    SELECT column_name, data_type
    FROM information_schema.columns
    WHERE table_name = '{tabela}';
    """
    return pd.read_sql(query, conn)

# Mostrando colunas de todas as tabelas
for table in tables:
    print(f"\n📄 Estrutura da tabela: {table}")
    print(listar_colunas(conn, table[0]))


📄 Estrutura da tabela: ('ibm_rfm',)
   column_name         data_type
0   cd_cliente  double precision
1     recencia            bigint
2   frequencia            bigint
3  valor_total  double precision
4  valor_medio  double precision

📄 Estrutura da tabela: ('ibm_familia',)


C:\Users\henri\AppData\Local\Temp\ipykernel_24680\1240962458.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)
C:\Users\henri\AppData\Local\Temp\ipykernel_24680\1240962458.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)


  column_name          data_type
0  cd_familia            numeric
1  nm_familia  character varying

📄 Estrutura da tabela: ('ibm_grupo',)
  column_name          data_type
0    cd_grupo            numeric
1  cd_familia            numeric
2    nm_grupo  character varying

📄 Estrutura da tabela: ('ibm_grupo_marca',)


C:\Users\henri\AppData\Local\Temp\ipykernel_24680\1240962458.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)
C:\Users\henri\AppData\Local\Temp\ipykernel_24680\1240962458.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)


  column_name data_type
0    cd_grupo   numeric
1    cd_marca   numeric

📄 Estrutura da tabela: ('ibm_marca',)
  column_name          data_type
0    cd_marca            numeric
1    nm_marca  character varying

📄 Estrutura da tabela: ('ibm_produto',)


...

C:\Users\henri\AppData\Local\Temp\ipykernel_24680\1240962458.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)
C:\Users\henri\AppData\Local\Temp\ipykernel_24680\1240962458.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)


   column_name data_type
0      nr_nota   numeric
1   dt_emissao      date
2   cd_cliente   numeric
3  cd_vendedor   numeric
4  cd_operacao   numeric
5     vl_total   numeric

📄 Estrutura da tabela: ('ibm_operacao',)
   column_name          data_type
0  cd_operacao            numeric
1  tp_operacao  character varying

📄 Estrutura da tabela: ('ibm_vendedor',)


C:\Users\henri\AppData\Local\Temp\ipykernel_24680\1240962458.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)
C:\Users\henri\AppData\Local\Temp\ipykernel_24680\1240962458.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)


   column_name          data_type
0  cd_vendedor            numeric
1  nm_vendedor  character varying

📄 Estrutura da tabela: ('modelos_ml',)
  column_name                    data_type
0          id                      integer
1   criado_em  timestamp without time zone
2      modelo                        bytea
3   nm_modelo            character varying
4      versao            character varying


C:\Users\henri\AppData\Local\Temp\ipykernel_24680\1240962458.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)


In [ ]:
# Função para atualizar a tabela RFM via SQL
def atualizar_rfm():
    conn = psycopg2.connect(
        host="SEU_HOST",
        database="SEU_DB",
        user="SEU_USER",
        password="SUA_SENHA",
        port=5432
    )
    cursor = conn.cursor()
    cursor.execute(open("/caminho/para/seu_script_rfm.sql", "r").read())
    conn.commit()
    cursor.close()
    conn.close()

# Função para rodar o modelo de churn prediction
def atualizar_modelo_churn():
    conn = psycopg2.connect(
        host="SEU_HOST",
        database="SEU_DB",
        user="SEU_USER",
        password="SUA_SENHA",
        port=5432
    )
    cursor = conn.cursor()
    cursor.execute("""
        SELECT modelo FROM modelos_ml
        WHERE nome_modelo = %s AND versao = %s
    """, ("catboost_churn", "v1"))
    
    modelo_bytes = cursor.fetchone()[0]
    modelo = joblib.load(io.BytesIO(modelo_bytes))
    
    df = pd.read_sql("SELECT * FROM dados_clientes_rfm", conn)
    predicoes = modelo.predict_proba(df)[:, 1]
    
    # Atualizando resultados no banco
    for idx, prob in enumerate(predicoes):
        cursor.execute("""
            UPDATE clientes SET prob_churn = %s WHERE id_cliente = %s
        """, (float(prob), df.iloc[idx]["id_cliente"]))
    
    conn.commit()
    cursor.close()
    conn.close()

# Função para rodar o modelo de clustering
def atualizar_clusterizacao():
    conn = psycopg2.connect(
        host="SEU_HOST",
        database="SEU_DB",
        user="SEU_USER",
        password="SUA_SENHA",
        port=5432
    )
    cursor = conn.cursor()
    cursor.execute("""
        SELECT modelo FROM modelos_ml
        WHERE nome_modelo = %s AND versao = %s
    """, ("kmeans_segmentacao", "v1"))
    
    modelo_bytes = cursor.fetchone()[0]
    modelo = joblib.load(io.BytesIO(modelo_bytes))
    
    df = pd.read_sql("SELECT * FROM dados_clientes_clusterizacao", conn)
    clusters = modelo.predict(df)
    
    for idx, grupo in enumerate(clusters):
        cursor.execute("""
            UPDATE clientes SET cluster = %s WHERE id_cliente = %s
        """, (int(grupo), df.iloc[idx]["id_cliente"]))

    conn.commit()
    cursor.close()
    conn.close()

# Configuração da DAG
default_args = {
    'owner': 'grupo55',
    'depends_on_past': False,
    'retries': 3,
    'retry_delay': timedelta(minutes=5),
    'start_date': datetime(2023, 1, 1),
}

dag = DAG(
    'pipeline_fidelidade_semanal',
    default_args=default_args,
    schedule_interval='@weekly',
    catchup=False
)

# Definindo as tarefas
t1 = PythonOperator(task_id='atualizar_rfm', python_callable=atualizar_rfm, dag=dag)
t2 = PythonOperator(task_id='atualizar_modelo_churn', python_callable=atualizar_modelo_churn, dag=dag)
t3 = PythonOperator(task_id='atualizar_clusterizacao', python_callable=atualizar_clusterizacao, dag=dag)

# Exemplo de notificação (ajuste os parâmetros conforme necessário)
t4 = EmailOperator(
    task_id='notificar_time',
    to='sua-equipe@example.com',
    subject='Pipeline semanal executado com sucesso',
    html_content='Todas as tarefas foram concluídas com sucesso!',
    dag=dag
)

# Definindo a ordem de execução
t1 >> [t2, t3] >> t4